In [1]:
import pandas as pd
import os
from tqdm import tqdm
import numpy as np
import re
import geopandas as gpd
import pickle
import yaml
import pickle

import warnings
warnings.filterwarnings('ignore')

In [7]:
data = pd.read_parquet("../../data/02_interim/sample_11_02_all_features_high_radius.parquet")

In [2]:
data = pd.read_parquet("../data/sample_11_02_all_features_high_radius.parquet")

In [32]:
test_data = pd.read_parquet('../data/second_departure_parquet_feats/sample_06.09.2022_all_features.parquet')

In [33]:
test_data

,date,datetime,dev_id_1,id_1_last_lon,id_1_last_lat,dev_id_2,id_2_last_lon,id_2_last_lat,accelerometer_x_corr,accelerometer_x_diff_corr,...,bearing_corr,speed_corr,height_diff_corr,bearing_diff_corr,speed_diff_corr,speed_mean_abs_diff,speed_mean_diff,speed_std_diff,distance_mean,distance_std
0,2022-09-06,2022-09-06 07:38:07,1CB8D3E9-04BE-465D-B334-0E1F7F026A98,37.602788,55.893562,3744EED3-A474-458B-AA4F-5357B8EE7FBF,37.606185,55.894089,0.000661,0.035777,...,0.205053,0.082422,0.014212,0.098073,0.029655,3.504287,-1.845214,9.843380,165.717646,141.099882
1,2022-09-06,2022-09-06 07:38:07,1CB8D3E9-04BE-465D-B334-0E1F7F026A98,37.602788,55.893562,393B6105-4D72-4E11-BEA1-394CD0AFC499,37.604149,55.895599,0.029039,0.067258,...,0.144063,0.052211,0.011149,0.152548,0.059629,2.732987,-1.196340,6.964679,133.042537,72.842306
2,2022-09-06,2022-09-06 07:38:07,3744EED3-A474-458B-AA4F-5357B8EE7FBF,37.606185,55.894089,393B6105-4D72-4E11-BEA1-394CD0AFC499,37.604149,55.895599,0.002194,0.077567,...,0.306874,0.012220,0.001329,0.043845,0.007309,4.445489,0.413304,11.253297,88.975281,98.468099
3,2022-09-06,2022-09-06 07:48:07,1CB8D3E9-04BE-465D-B334-0E1F7F026A98,37.602780,55.893606,7F921169-FB12-471F-B244-62840E543DD7,37.603513,55.892299,0.024953,0.069213,...,0.046329,0.009875,0.011567,0.269572,0.014251,3.261829,-0.938763,8.793781,118.765372,16.460189
4,2022-09-06,2022-09-06 08:13:07,40808752-2CEE-454B-B6E6-95EC9615999D,37.603324,55.892073,8A8813E7-6012-4600-835D-1C286EADE837,37.602763,55.892618,0.203208,0.009416,...,0.204592,0.112317,0.010256,0.058706,0.119389,3.765288,0.098938,10.963710,38.229802,39.078068
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1674,2022-09-06,2022-09-06 18:13:07,0699B415-A73E-4CFA-B910-079AA0408B1B,37.611471,55.886099,294A06F3-F7FE-4AAC-8099-049733BA1315,37.611448,55.886091,0.080755,0.157526,...,0.344557,0.074496,0.044965,0.090145,0.081442,2.609243,-0.734082,8.615557,15.131298,18.080598
1675,2022-09-06,2022-09-06 18:18:07,0699B415-A73E-4CFA-B910-079AA0408B1B,37.611899,55.886257,294A06F3-F7FE-4AAC-8099-049733BA1315,37.611894,55.886258,0.150419,0.276300,...,0.365747,0.009269,0.004239,0.016793,0.033839,1.568117,-0.382263,4.930226,12.228579,10.852700
1676,2022-09-06,2022-09-06 18:23:07,0699B415-A73E-4CFA-B910-079AA0408B1B,37.611934,55.886257,294A06F3-F7FE-4AAC-8099-049733BA1315,37.611912,55.886258,0.859881,0.028291,...,0.175229,0.015441,NaN,0.059281,0.006019,1.321455,0.411513,4.601880,9.306995,10.154824
1677,2022-09-06,2022-09-06 18:23:07,24F61894-96E0-4B35-9E6E-A1B2F1614D35,37.613846,55.894238,2D7135B3-1208-45BA-9B5A-0FC27C13B9B0,37.614784,55.893775,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
test_data.columns == data.columns[:-4]

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True])

In [35]:
list_of_files = list()
for (dirpath, dirnames, filenames) in os.walk("../data/second_departure_parquet_feats/"):
    list_of_files += [os.path.join(dirpath, file) for file in filenames if ".parquet" in file]

In [36]:
list_of_files

['../data/second_departure_parquet_feats/sample_06.09.2022_all_features.parquet',
 '../data/second_departure_parquet_feats/sample_07.09.2022_all_features.parquet',
 '../data/second_departure_parquet_feats/sample_08.09.2022_all_features.parquet',
 '../data/second_departure_parquet_feats/sample_09.09.2022_all_features.parquet',
 '../data/second_departure_parquet_feats/sample_12.09.2022_all_features.parquet']

In [37]:
all_data = []
for filename in list_of_files:
    temp_data = pd.read_parquet(filename)
    all_data.append(temp_data)

In [38]:
new_df = pd.concat(all_data, ignore_index=True)

In [3]:
with open("../artifacts/random_forest_model.pkl", 'rb') as fin:
    model = pickle.load(fin)
    
with open("../artifacts/features.yaml", 'r') as fin:
    features = yaml.safe_load(fin)

In [4]:
features

['accelerometer_x_corr',
 'accelerometer_x_diff_corr',
 'accelerometer_y_corr',
 'accelerometer_y_diff_corr',
 'accelerometer_z_corr',
 'accelerometer_z_diff_corr',
 'x_metres_corr',
 'x_diff_metres_corr',
 'y_metres_corr',
 'y_diff_metres_corr',
 'height_corr',
 'bearing_corr',
 'speed_corr',
 'height_diff_corr',
 'bearing_diff_corr',
 'speed_diff_corr',
 'speed_mean_abs_diff',
 'speed_mean_diff',
 'speed_std_diff',
 'distance_mean',
 'distance_std']

In [5]:
data

,date,datetime,dev_id_1,id_1_last_lon,id_1_last_lat,dev_id_2,id_2_last_lon,id_2_last_lat,accelerometer_x_corr,accelerometer_x_diff_corr,...,bearing_corr,speed_corr,height_diff_corr,bearing_diff_corr,speed_diff_corr,speed_mean_abs_diff,speed_mean_diff,speed_std_diff,distance_mean,distance_std
0,2022-12-01,2022-12-01 10:48:18,D4909E26-0469-4045-B426-211006C38915,37.613123,55.894199,E9B9C64C-B66D-416D-BACD-3F0C9C2E1767,37.597539,55.891541,0.020848,0.013961,...,0.003843,0.040824,2.342438e-18,0.132480,0.033372,1.394553,0.712902,4.810753,2054.569542,19.910991
1,2022-12-01,2022-12-01 10:53:18,D4909E26-0469-4045-B426-211006C38915,37.613346,55.894110,E9B9C64C-B66D-416D-BACD-3F0C9C2E1767,37.597614,55.891387,0.024866,0.003584,...,0.045900,0.044286,3.524988e-02,0.002435,0.019051,2.427950,0.921701,6.956017,2037.886971,28.447167
2,2022-12-01,2022-12-01 10:58:18,D4909E26-0469-4045-B426-211006C38915,37.613084,55.893969,E9B9C64C-B66D-416D-BACD-3F0C9C2E1767,37.597684,55.891430,0.019898,0.030826,...,0.089580,0.078751,2.599786e-19,0.000066,0.108915,2.373555,1.524765,7.726640,2030.144925,26.966362
3,2022-12-01,2022-12-01 11:03:18,D4909E26-0469-4045-B426-211006C38915,37.613346,55.894205,E9B9C64C-B66D-416D-BACD-3F0C9C2E1767,37.597931,55.889485,0.030749,0.033406,...,0.154146,0.058034,3.337160e-03,0.037221,0.062985,2.975291,0.145192,7.078031,2010.692280,34.467854
4,2022-12-01,2022-12-01 11:08:18,D4909E26-0469-4045-B426-211006C38915,37.613375,55.894186,E9B9C64C-B66D-416D-BACD-3F0C9C2E1767,37.599243,55.889024,0.063623,0.093234,...,0.057259,0.059225,2.638958e-03,0.000309,0.074824,1.477277,-0.082662,3.710233,1936.012978,54.545285
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
478,2022-11-29,2022-11-29 16:36:43,D4909E26-0469-4045-B426-211006C38915,37.613723,55.893935,E9B9C64C-B66D-416D-BACD-3F0C9C2E1767,37.605537,55.889927,0.002929,0.000234,...,0.026805,0.044368,2.634802e-19,0.057983,0.052032,2.012396,-0.194358,5.153748,1137.248254,14.953325
479,2022-11-29,2022-11-29 16:41:43,D4909E26-0469-4045-B426-211006C38915,37.612643,55.893774,E9B9C64C-B66D-416D-BACD-3F0C9C2E1767,37.605575,55.889784,0.018344,0.054240,...,0.187970,0.033988,3.013717e-19,0.001197,0.001480,1.757378,0.350348,4.671373,1064.982394,65.319743
480,2022-11-29,2022-11-29 16:46:43,D4909E26-0469-4045-B426-211006C38915,37.612672,55.893756,E9B9C64C-B66D-416D-BACD-3F0C9C2E1767,37.605116,55.889897,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
481,2022-11-29,2022-11-29 17:21:43,D4909E26-0469-4045-B426-211006C38915,37.612819,55.893607,E9B9C64C-B66D-416D-BACD-3F0C9C2E1767,37.603393,55.889129,0.044405,0.014835,...,0.220607,0.072323,2.309347e-19,0.004155,0.036093,3.226823,0.081517,8.832679,1311.463707,23.769500


In [6]:
data.loc[data["dev_id_1"].str[-1] == data["dev_id_2"].str[-1], 'target'] = 1

In [39]:
new_df.loc[new_df["dev_id_1"].str[-1] == new_df["dev_id_2"].str[-1], 'target'] = 1

In [7]:
data

,date,datetime,dev_id_1,id_1_last_lon,id_1_last_lat,dev_id_2,id_2_last_lon,id_2_last_lat,accelerometer_x_corr,accelerometer_x_diff_corr,...,speed_corr,height_diff_corr,bearing_diff_corr,speed_diff_corr,speed_mean_abs_diff,speed_mean_diff,speed_std_diff,distance_mean,distance_std,target
0,2022-12-01,2022-12-01 10:48:18,D4909E26-0469-4045-B426-211006C38915,37.613123,55.894199,E9B9C64C-B66D-416D-BACD-3F0C9C2E1767,37.597539,55.891541,0.020848,0.013961,...,0.040824,2.342438e-18,0.132480,0.033372,1.394553,0.712902,4.810753,2054.569542,19.910991,NaN
1,2022-12-01,2022-12-01 10:53:18,D4909E26-0469-4045-B426-211006C38915,37.613346,55.894110,E9B9C64C-B66D-416D-BACD-3F0C9C2E1767,37.597614,55.891387,0.024866,0.003584,...,0.044286,3.524988e-02,0.002435,0.019051,2.427950,0.921701,6.956017,2037.886971,28.447167,NaN
2,2022-12-01,2022-12-01 10:58:18,D4909E26-0469-4045-B426-211006C38915,37.613084,55.893969,E9B9C64C-B66D-416D-BACD-3F0C9C2E1767,37.597684,55.891430,0.019898,0.030826,...,0.078751,2.599786e-19,0.000066,0.108915,2.373555,1.524765,7.726640,2030.144925,26.966362,NaN
3,2022-12-01,2022-12-01 11:03:18,D4909E26-0469-4045-B426-211006C38915,37.613346,55.894205,E9B9C64C-B66D-416D-BACD-3F0C9C2E1767,37.597931,55.889485,0.030749,0.033406,...,0.058034,3.337160e-03,0.037221,0.062985,2.975291,0.145192,7.078031,2010.692280,34.467854,NaN
4,2022-12-01,2022-12-01 11:08:18,D4909E26-0469-4045-B426-211006C38915,37.613375,55.894186,E9B9C64C-B66D-416D-BACD-3F0C9C2E1767,37.599243,55.889024,0.063623,0.093234,...,0.059225,2.638958e-03,0.000309,0.074824,1.477277,-0.082662,3.710233,1936.012978,54.545285,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
478,2022-11-29,2022-11-29 16:36:43,D4909E26-0469-4045-B426-211006C38915,37.613723,55.893935,E9B9C64C-B66D-416D-BACD-3F0C9C2E1767,37.605537,55.889927,0.002929,0.000234,...,0.044368,2.634802e-19,0.057983,0.052032,2.012396,-0.194358,5.153748,1137.248254,14.953325,NaN
479,2022-11-29,2022-11-29 16:41:43,D4909E26-0469-4045-B426-211006C38915,37.612643,55.893774,E9B9C64C-B66D-416D-BACD-3F0C9C2E1767,37.605575,55.889784,0.018344,0.054240,...,0.033988,3.013717e-19,0.001197,0.001480,1.757378,0.350348,4.671373,1064.982394,65.319743,NaN
480,2022-11-29,2022-11-29 16:46:43,D4909E26-0469-4045-B426-211006C38915,37.612672,55.893756,E9B9C64C-B66D-416D-BACD-3F0C9C2E1767,37.605116,55.889897,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
481,2022-11-29,2022-11-29 17:21:43,D4909E26-0469-4045-B426-211006C38915,37.612819,55.893607,E9B9C64C-B66D-416D-BACD-3F0C9C2E1767,37.603393,55.889129,0.044405,0.014835,...,0.072323,2.309347e-19,0.004155,0.036093,3.226823,0.081517,8.832679,1311.463707,23.769500,NaN


In [8]:
data['target'] = data['target'].fillna(0)

In [40]:
new_df['target'] = new_df['target'].fillna(0)

In [9]:
data

,date,datetime,dev_id_1,id_1_last_lon,id_1_last_lat,dev_id_2,id_2_last_lon,id_2_last_lat,accelerometer_x_corr,accelerometer_x_diff_corr,...,speed_corr,height_diff_corr,bearing_diff_corr,speed_diff_corr,speed_mean_abs_diff,speed_mean_diff,speed_std_diff,distance_mean,distance_std,target
0,2022-12-01,2022-12-01 10:48:18,D4909E26-0469-4045-B426-211006C38915,37.613123,55.894199,E9B9C64C-B66D-416D-BACD-3F0C9C2E1767,37.597539,55.891541,0.020848,0.013961,...,0.040824,2.342438e-18,0.132480,0.033372,1.394553,0.712902,4.810753,2054.569542,19.910991,0.0
1,2022-12-01,2022-12-01 10:53:18,D4909E26-0469-4045-B426-211006C38915,37.613346,55.894110,E9B9C64C-B66D-416D-BACD-3F0C9C2E1767,37.597614,55.891387,0.024866,0.003584,...,0.044286,3.524988e-02,0.002435,0.019051,2.427950,0.921701,6.956017,2037.886971,28.447167,0.0
2,2022-12-01,2022-12-01 10:58:18,D4909E26-0469-4045-B426-211006C38915,37.613084,55.893969,E9B9C64C-B66D-416D-BACD-3F0C9C2E1767,37.597684,55.891430,0.019898,0.030826,...,0.078751,2.599786e-19,0.000066,0.108915,2.373555,1.524765,7.726640,2030.144925,26.966362,0.0
3,2022-12-01,2022-12-01 11:03:18,D4909E26-0469-4045-B426-211006C38915,37.613346,55.894205,E9B9C64C-B66D-416D-BACD-3F0C9C2E1767,37.597931,55.889485,0.030749,0.033406,...,0.058034,3.337160e-03,0.037221,0.062985,2.975291,0.145192,7.078031,2010.692280,34.467854,0.0
4,2022-12-01,2022-12-01 11:08:18,D4909E26-0469-4045-B426-211006C38915,37.613375,55.894186,E9B9C64C-B66D-416D-BACD-3F0C9C2E1767,37.599243,55.889024,0.063623,0.093234,...,0.059225,2.638958e-03,0.000309,0.074824,1.477277,-0.082662,3.710233,1936.012978,54.545285,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
478,2022-11-29,2022-11-29 16:36:43,D4909E26-0469-4045-B426-211006C38915,37.613723,55.893935,E9B9C64C-B66D-416D-BACD-3F0C9C2E1767,37.605537,55.889927,0.002929,0.000234,...,0.044368,2.634802e-19,0.057983,0.052032,2.012396,-0.194358,5.153748,1137.248254,14.953325,0.0
479,2022-11-29,2022-11-29 16:41:43,D4909E26-0469-4045-B426-211006C38915,37.612643,55.893774,E9B9C64C-B66D-416D-BACD-3F0C9C2E1767,37.605575,55.889784,0.018344,0.054240,...,0.033988,3.013717e-19,0.001197,0.001480,1.757378,0.350348,4.671373,1064.982394,65.319743,0.0
480,2022-11-29,2022-11-29 16:46:43,D4909E26-0469-4045-B426-211006C38915,37.612672,55.893756,E9B9C64C-B66D-416D-BACD-3F0C9C2E1767,37.605116,55.889897,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
481,2022-11-29,2022-11-29 17:21:43,D4909E26-0469-4045-B426-211006C38915,37.612819,55.893607,E9B9C64C-B66D-416D-BACD-3F0C9C2E1767,37.603393,55.889129,0.044405,0.014835,...,0.072323,2.309347e-19,0.004155,0.036093,3.226823,0.081517,8.832679,1311.463707,23.769500,0.0


In [10]:
data['predicted_proba'] = model.predict_proba(data[features].fillna(-999))[:, 1]

In [41]:
new_df['predicted_proba'] = model.predict_proba(new_df[features].fillna(-999))[:, 1]

In [11]:
data

,date,datetime,dev_id_1,id_1_last_lon,id_1_last_lat,dev_id_2,id_2_last_lon,id_2_last_lat,accelerometer_x_corr,accelerometer_x_diff_corr,...,height_diff_corr,bearing_diff_corr,speed_diff_corr,speed_mean_abs_diff,speed_mean_diff,speed_std_diff,distance_mean,distance_std,target,predicted_proba
0,2022-12-01,2022-12-01 10:48:18,D4909E26-0469-4045-B426-211006C38915,37.613123,55.894199,E9B9C64C-B66D-416D-BACD-3F0C9C2E1767,37.597539,55.891541,0.020848,0.013961,...,2.342438e-18,0.132480,0.033372,1.394553,0.712902,4.810753,2054.569542,19.910991,0.0,0.032000
1,2022-12-01,2022-12-01 10:53:18,D4909E26-0469-4045-B426-211006C38915,37.613346,55.894110,E9B9C64C-B66D-416D-BACD-3F0C9C2E1767,37.597614,55.891387,0.024866,0.003584,...,3.524988e-02,0.002435,0.019051,2.427950,0.921701,6.956017,2037.886971,28.447167,0.0,0.022000
2,2022-12-01,2022-12-01 10:58:18,D4909E26-0469-4045-B426-211006C38915,37.613084,55.893969,E9B9C64C-B66D-416D-BACD-3F0C9C2E1767,37.597684,55.891430,0.019898,0.030826,...,2.599786e-19,0.000066,0.108915,2.373555,1.524765,7.726640,2030.144925,26.966362,0.0,0.034000
3,2022-12-01,2022-12-01 11:03:18,D4909E26-0469-4045-B426-211006C38915,37.613346,55.894205,E9B9C64C-B66D-416D-BACD-3F0C9C2E1767,37.597931,55.889485,0.030749,0.033406,...,3.337160e-03,0.037221,0.062985,2.975291,0.145192,7.078031,2010.692280,34.467854,0.0,0.006000
4,2022-12-01,2022-12-01 11:08:18,D4909E26-0469-4045-B426-211006C38915,37.613375,55.894186,E9B9C64C-B66D-416D-BACD-3F0C9C2E1767,37.599243,55.889024,0.063623,0.093234,...,2.638958e-03,0.000309,0.074824,1.477277,-0.082662,3.710233,1936.012978,54.545285,0.0,0.030000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
478,2022-11-29,2022-11-29 16:36:43,D4909E26-0469-4045-B426-211006C38915,37.613723,55.893935,E9B9C64C-B66D-416D-BACD-3F0C9C2E1767,37.605537,55.889927,0.002929,0.000234,...,2.634802e-19,0.057983,0.052032,2.012396,-0.194358,5.153748,1137.248254,14.953325,0.0,0.052000
479,2022-11-29,2022-11-29 16:41:43,D4909E26-0469-4045-B426-211006C38915,37.612643,55.893774,E9B9C64C-B66D-416D-BACD-3F0C9C2E1767,37.605575,55.889784,0.018344,0.054240,...,3.013717e-19,0.001197,0.001480,1.757378,0.350348,4.671373,1064.982394,65.319743,0.0,0.004000
480,2022-11-29,2022-11-29 16:46:43,D4909E26-0469-4045-B426-211006C38915,37.612672,55.893756,E9B9C64C-B66D-416D-BACD-3F0C9C2E1767,37.605116,55.889897,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.150476
481,2022-11-29,2022-11-29 17:21:43,D4909E26-0469-4045-B426-211006C38915,37.612819,55.893607,E9B9C64C-B66D-416D-BACD-3F0C9C2E1767,37.603393,55.889129,0.044405,0.014835,...,2.309347e-19,0.004155,0.036093,3.226823,0.081517,8.832679,1311.463707,23.769500,0.0,0.070000


In [12]:
data['thresh'] = 0.416

In [42]:
new_df['thresh'] = 0.416

In [13]:
data

,date,datetime,dev_id_1,id_1_last_lon,id_1_last_lat,dev_id_2,id_2_last_lon,id_2_last_lat,accelerometer_x_corr,accelerometer_x_diff_corr,...,bearing_diff_corr,speed_diff_corr,speed_mean_abs_diff,speed_mean_diff,speed_std_diff,distance_mean,distance_std,target,predicted_proba,thresh
0,2022-12-01,2022-12-01 10:48:18,D4909E26-0469-4045-B426-211006C38915,37.613123,55.894199,E9B9C64C-B66D-416D-BACD-3F0C9C2E1767,37.597539,55.891541,0.020848,0.013961,...,0.132480,0.033372,1.394553,0.712902,4.810753,2054.569542,19.910991,0.0,0.032000,0.416
1,2022-12-01,2022-12-01 10:53:18,D4909E26-0469-4045-B426-211006C38915,37.613346,55.894110,E9B9C64C-B66D-416D-BACD-3F0C9C2E1767,37.597614,55.891387,0.024866,0.003584,...,0.002435,0.019051,2.427950,0.921701,6.956017,2037.886971,28.447167,0.0,0.022000,0.416
2,2022-12-01,2022-12-01 10:58:18,D4909E26-0469-4045-B426-211006C38915,37.613084,55.893969,E9B9C64C-B66D-416D-BACD-3F0C9C2E1767,37.597684,55.891430,0.019898,0.030826,...,0.000066,0.108915,2.373555,1.524765,7.726640,2030.144925,26.966362,0.0,0.034000,0.416
3,2022-12-01,2022-12-01 11:03:18,D4909E26-0469-4045-B426-211006C38915,37.613346,55.894205,E9B9C64C-B66D-416D-BACD-3F0C9C2E1767,37.597931,55.889485,0.030749,0.033406,...,0.037221,0.062985,2.975291,0.145192,7.078031,2010.692280,34.467854,0.0,0.006000,0.416
4,2022-12-01,2022-12-01 11:08:18,D4909E26-0469-4045-B426-211006C38915,37.613375,55.894186,E9B9C64C-B66D-416D-BACD-3F0C9C2E1767,37.599243,55.889024,0.063623,0.093234,...,0.000309,0.074824,1.477277,-0.082662,3.710233,1936.012978,54.545285,0.0,0.030000,0.416
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
478,2022-11-29,2022-11-29 16:36:43,D4909E26-0469-4045-B426-211006C38915,37.613723,55.893935,E9B9C64C-B66D-416D-BACD-3F0C9C2E1767,37.605537,55.889927,0.002929,0.000234,...,0.057983,0.052032,2.012396,-0.194358,5.153748,1137.248254,14.953325,0.0,0.052000,0.416
479,2022-11-29,2022-11-29 16:41:43,D4909E26-0469-4045-B426-211006C38915,37.612643,55.893774,E9B9C64C-B66D-416D-BACD-3F0C9C2E1767,37.605575,55.889784,0.018344,0.054240,...,0.001197,0.001480,1.757378,0.350348,4.671373,1064.982394,65.319743,0.0,0.004000,0.416
480,2022-11-29,2022-11-29 16:46:43,D4909E26-0469-4045-B426-211006C38915,37.612672,55.893756,E9B9C64C-B66D-416D-BACD-3F0C9C2E1767,37.605116,55.889897,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.150476,0.416
481,2022-11-29,2022-11-29 17:21:43,D4909E26-0469-4045-B426-211006C38915,37.612819,55.893607,E9B9C64C-B66D-416D-BACD-3F0C9C2E1767,37.603393,55.889129,0.044405,0.014835,...,0.004155,0.036093,3.226823,0.081517,8.832679,1311.463707,23.769500,0.0,0.070000,0.416


In [14]:
data['label'] = (data['predicted_proba'] >= data['thresh']).astype(int)

In [43]:
new_df['label'] = (new_df['predicted_proba'] >= new_df['thresh']).astype(int)

In [15]:
data

,date,datetime,dev_id_1,id_1_last_lon,id_1_last_lat,dev_id_2,id_2_last_lon,id_2_last_lat,accelerometer_x_corr,accelerometer_x_diff_corr,...,speed_diff_corr,speed_mean_abs_diff,speed_mean_diff,speed_std_diff,distance_mean,distance_std,target,predicted_proba,thresh,label
0,2022-12-01,2022-12-01 10:48:18,D4909E26-0469-4045-B426-211006C38915,37.613123,55.894199,E9B9C64C-B66D-416D-BACD-3F0C9C2E1767,37.597539,55.891541,0.020848,0.013961,...,0.033372,1.394553,0.712902,4.810753,2054.569542,19.910991,0.0,0.032000,0.416,0
1,2022-12-01,2022-12-01 10:53:18,D4909E26-0469-4045-B426-211006C38915,37.613346,55.894110,E9B9C64C-B66D-416D-BACD-3F0C9C2E1767,37.597614,55.891387,0.024866,0.003584,...,0.019051,2.427950,0.921701,6.956017,2037.886971,28.447167,0.0,0.022000,0.416,0
2,2022-12-01,2022-12-01 10:58:18,D4909E26-0469-4045-B426-211006C38915,37.613084,55.893969,E9B9C64C-B66D-416D-BACD-3F0C9C2E1767,37.597684,55.891430,0.019898,0.030826,...,0.108915,2.373555,1.524765,7.726640,2030.144925,26.966362,0.0,0.034000,0.416,0
3,2022-12-01,2022-12-01 11:03:18,D4909E26-0469-4045-B426-211006C38915,37.613346,55.894205,E9B9C64C-B66D-416D-BACD-3F0C9C2E1767,37.597931,55.889485,0.030749,0.033406,...,0.062985,2.975291,0.145192,7.078031,2010.692280,34.467854,0.0,0.006000,0.416,0
4,2022-12-01,2022-12-01 11:08:18,D4909E26-0469-4045-B426-211006C38915,37.613375,55.894186,E9B9C64C-B66D-416D-BACD-3F0C9C2E1767,37.599243,55.889024,0.063623,0.093234,...,0.074824,1.477277,-0.082662,3.710233,1936.012978,54.545285,0.0,0.030000,0.416,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
478,2022-11-29,2022-11-29 16:36:43,D4909E26-0469-4045-B426-211006C38915,37.613723,55.893935,E9B9C64C-B66D-416D-BACD-3F0C9C2E1767,37.605537,55.889927,0.002929,0.000234,...,0.052032,2.012396,-0.194358,5.153748,1137.248254,14.953325,0.0,0.052000,0.416,0
479,2022-11-29,2022-11-29 16:41:43,D4909E26-0469-4045-B426-211006C38915,37.612643,55.893774,E9B9C64C-B66D-416D-BACD-3F0C9C2E1767,37.605575,55.889784,0.018344,0.054240,...,0.001480,1.757378,0.350348,4.671373,1064.982394,65.319743,0.0,0.004000,0.416,0
480,2022-11-29,2022-11-29 16:46:43,D4909E26-0469-4045-B426-211006C38915,37.612672,55.893756,E9B9C64C-B66D-416D-BACD-3F0C9C2E1767,37.605116,55.889897,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.150476,0.416,0
481,2022-11-29,2022-11-29 17:21:43,D4909E26-0469-4045-B426-211006C38915,37.612819,55.893607,E9B9C64C-B66D-416D-BACD-3F0C9C2E1767,37.603393,55.889129,0.044405,0.014835,...,0.036093,3.226823,0.081517,8.832679,1311.463707,23.769500,0.0,0.070000,0.416,0


In [16]:
data['target'].value_counts()

target
0.0    426
1.0     57
Name: count, dtype: int64

In [44]:
new_df['target'].value_counts()

target
0.0    6299
1.0     253
Name: count, dtype: int64

In [17]:
data.shape[0]

483

In [45]:
new_df.shape[0]

6552

In [46]:
(new_df['target'] == new_df['label']).sum()

5320

In [47]:
5320/6552

0.811965811965812

In [43]:
477 / 483

0.9875776397515528

In [44]:
data.to_csv("../../data/05_analytical_report_new/02_anomaly_detection.csv", index=False)

In [18]:
(data['target'] == data['label']).sum()

477

In [37]:
data["label"].value_counts()

0    430
1     53
Name: label, dtype: int64

In [29]:
data['target']

0      0.0
1      0.0
2      0.0
3      0.0
4      0.0
      ... 
478    0.0
479    0.0
480    0.0
481    0.0
482    0.0
Name: target, Length: 483, dtype: float64

In [19]:
data["dev_id_1"].unique()

array(['D4909E26-0469-4045-B426-211006C38915',
       'A1303E56-0163-1015-B156-511006C38315'], dtype=object)

In [22]:
(model.predict_proba(data[features].fillna(-999))[:, 1] > 0.416).sum()

53

In [ ]:
data["predict_proba"]

In [6]:
data["id_1_last_geometry"].isna().sum()

6552